# 📘 About

This Python script interacts with NASA's **EPIC (Earth Polychromatic Imaging Camera) Daily “Blue Marble” API** to automate the process of downloading high-resolution satellite imagery of Earth.

The script:

- Retrieves all available dates with EPIC imagery.
- Fetches metadata for a specific date.
- Extracts image identifiers and builds direct download URLs.
- Downloads and saves the images locally using their timestamp-based identifier.

It uses the **natural color** version of the EPIC image archive.

🔗 **API Documentation:**  
[https://epic.gsfc.nasa.gov/about/api](https://epic.gsfc.nasa.gov/about/api)

🖼️ **Sample Image Preview:**  
![EPIC Sample](https://epic.gsfc.nasa.gov/archive/natural/2015/10/31/png/epic_1b_20151031074844.png)


# Environment

**Make you to use the requirements.txt in your environment**

In [ ]:
import os
import requests
from datetime import datetime
import pandas as pd 

# CONSTANS and Variables

In [ ]:
# Base endpoints
MAIN_URL      = "https://epic.gsfc.nasa.gov/api/natural"
MAIN_PATH     = "ADD_THE_PATH_TO_DOWNLOAD_IMAGES"
PATH_FILE     = "ADD_THE_PATH_TO_SAVE_AN_EXCEL_FILE"      # eg: 'your_path/your_file.xlsx'

os.chdir(MAIN_PATH)

# Funtions 

In [ ]:


def get_available_dates():
    """Returns a list of strings like '2025-07-01', etc."""
    resp = requests.get(MAIN_URL + "/all")
    resp.raise_for_status()
    return resp.json()

def get_metadata_for_date(date_str):
    """Fetches the list of image metadata for a particular date."""
    # date_str must be 'YYYY-MM-DD'
    MAIN_URL_MODIFIED = MAIN_URL.replace('natural','enhanced')
    resp = requests.get(f"{MAIN_URL_MODIFIED}/date/{date_str}")
    resp.raise_for_status()
    return resp.json()

def format_request(list_identifiers):
    """Using a list creates a df and creates the structure of the request."""
    d = {'identifier': list_identifiers}
    df = pd.DataFrame(data=d)
    print(df.shape)
    df = df.drop_duplicates()
    print(df.shape)  # There can be some duplicates

    df['year'] = df['identifier'].apply(lambda x: x[0:4])
    df['month'] = df['identifier'].apply(lambda x: x[4:6])
    df['day'] = df['identifier'].apply(lambda x: x[6:8])
    df['request'] = 'https://epic.gsfc.nasa.gov/archive/natural/'+df['year']+'/'+df['month']+'/'+df['day']+'/png/epic_1b_'+df['identifier']+'.png'
    return df 

def download_image(df):
    """Using the df request the urls of the images, then saves each image with the identifier"""
        for i in range(0,len(df)+1):
            try: 
                picture_request = requests.get(f'{df['request'].iloc[i]}')  
                if picture_request.status_code == 200:
                    with open(f"{df['identifier'].iloc[i]}.png", 'wb') as img:  
                        img.write(picture_request.content)
            except Exception as e: 
                print('---------------------------------------')
                print(f'Error in url: {df['request'].iloc[i]}')
                print(f'Error description: {e}')
                pass 

# Code Execution

In [ ]:
if __name__ == "__main__":
    
    # Step 1 
    dates = get_available_dates()
    list_identifiers = []
    
    # Step 2
    for i in dates:
        date_view = i['date']
        meta = get_metadata_for_date(date_view)
        for m in meta:
            identifier = m["identifier"]
            list_identifiers.append(identifier)

    # Step 3
    df = format_request(list_identifiers)
    try: 
        df.to_excel(f'{PATH_FILE}', index = False)
    except: 
        print("No se pudo exportar el excel")
    
    # Step 4
    download_image(df)